# State-specific Choice Sets

This notebook discusses how we want to implement state spaces which are aware of dense dimension and flexible choice sets.

## Design
A range of considerations have entered the discussion of how the state space should be adapted to allow for flexible choice_sets in an efficient and tractable way.   Initially we agreed that `SubStateSpaces` could be the key building blocks of a revised `StateSpace`. 
In this approach a `StateSpace` consists of multiple `SubStateSpace`s for each combination of dense dimension vector, period, and choice set.

While it is clear that we need one object that carries information about each dense, period, choice-set combination I do not think that `SubStateSpace`s in the original sense should be the base of the new state space. 
Due to the fact that time is the variable we optimize and iterate the model over, I came to the conlusion that it is most natural to bundle methods and information over periods.
If we would define seperate objects for each period we would define many objects that essentially do the same. 
While that would be possible it would require more repetitive code and would moreover 
not be any more flexible than the design we have now. 

Essentially this is due to the fact that we still want to do calculate several things for the whole core before we initialize any dense specific sps!

In our model we know that dense dimensions duplicate the state space and choice sets seperate the state space. 
The handling of the dense variables has not substantially changed from the case without flexible choice sets.

That is why we can start by generating all general core information and defining a class that intializes indiviudal state spaces for each dense dimension. 
These indiviudal state spaces can now be thought of one core that can be split in subcores with different information. 
Each of this subcores will require different objects and calculations in the model solution. 
That is why we generate an individual class for each choice set.
In principal that is not much different than just adding one level to the state space construction that we had before. 
The crucial difference however is that ChoiceSpaces have to communicate with each other in contrast to cores with different dense dims. 
Much information such as shocks or wages only concern this ChoiceSetSpace and can be saved privately. 
Some information such as the continuation values have to be saved on the whole core since transitions between choice sets over periods are possible. 

The base for creating this StateSpace is a dict that contains (period,choice_set,dense): core_indices as items. 
Most of the operations will look fairly similair and it seems like most of the 
changes fit in fairly smooth. 

The exact architecture looks follows:
Additional to the classes before we introduce a _ChoiceSetSpace that is called for each choice_set by the _SingleDimStateSpace. 
In the pre processing state we will create a dict with {(period, choice_set, dense_dim):core index}. 
This will be used to distribute core sections from _SingleDimStateSpace to _ChoiceSetSpaces.
Many calculations like child indices or covariates will be done as before. 
The set attribute method changes since we have a method to save information in _SinglDImStateSpaces and one to save infomration in _ChoiceSetSpaces now!

In [1]:
import numpy as np
import pandas as pd
import respy as rp
import itertools
import copy

from respy.pre_processing.model_processing import process_params_and_options
from respy.state_space import _create_core_and_indexer, create_state_space_class
from respy.pre_processing.process_covariates import separate_covariates_into_core_dense_mixed
from respy.shared import create_base_draws

### Load the example model

No exogenous processes for now.

In [18]:
# Load model.
params, options = rp.get_example_model("robinson_crusoe_extended", with_data=False)

# Extend with observable characteristic.
params.loc[("observable_health_well", "probability"), "value"] = 0.9
params.loc[("observable_health_sick", "probability"), "value"] = 0.1

# Sick people can never work.
options["inadmissible_choices"] = {
    "fishing": ["health == 'sick'"],
}

# Create internal specification objects.
optim_paras, options_ = process_params_and_options(params, options)


In [19]:
options_ = separate_covariates_into_core_dense_mixed(options_, optim_paras)

### Pre Processing

We want to split the inadmissability contraints in three parts. 
1. options["inadmissable_choices_core"]: 
These are choice restrictions that just depend on the core state! 
- We will apply these choices to create choice sets for each core state. 

2. options["inadmissable_choices_dense"]:
- We will create all dense combinations and we will assign all of these combinations admissable choice sets

3. options["inadmissable_choices_mixed"]
- We will write a function that transforms the mixed constraints to a constraint on a part of the core sp with a fixed dense comb! 

In [20]:
def separate_choice_restrictions_into_core_dense_mixed(options, optim_paras):
    """
    This is the simplest version we can possbly think of!
    #TODO: Align with separation of covariates. If possible unify both.
    If not try to synchronize as far as possible!
    """
    options = copy.deepcopy(options)
    covariates = options["covariates"]
    
    # Add ne dict keys
    constr_list = list()
    for choice in options["inadmissible_choices"].keys():
        for choice_constr in options["inadmissible_choices"][choice]:
            if any(x in choice_constr for x in options["covariates_dense"]) is False:
                constr_list.append((choice_constr, choice, "core"))    
            elif any(x in choice_constr for x in options["covariates_core"]) is False:
                constr_list.append((choice_constr, choice, "dense"))
            else:
                constr_list.append((choice_constr, choice, "mixed"))        
    
    for sp in ["core", "dense", "mixed"]:
        options[f"inadmissible_choices_{sp}"] = {}
        for choice in options["inadmissible_choices"].keys():
            relevant_contraints = [ x for x in constr_list if x[1]==choice and x[2]==sp]
            if relevant_contraints == []:
                pass
            else:
                options[f"inadmissible_choices_{sp}"][choice] = relevant_contraints
    return options



In [21]:
options_ = separate_choice_restrictions_into_core_dense_mixed(options_, optim_paras)

### Create object for the initialization of the sub state spaces

We want to define methods that alow us to create objects that we can 
base our state space construction upon! 

Therefore we will create the core Sp as before. We will use the choice restrictions to create a dict that contains a choice/period comb as keys and the corresponding part of the core sp df as value!

We will also apply constraints to the dense sp!

Therafter we will combine the information and apply mixed constraints. 
In the end we want a dict that contains (period, choice_set, dense_point): np.array([indexes])




In [48]:
def _create_choice_sets(df, optim_paras, options, category):
    """
    Todo: Check to which extent we can use the same procedure for 
    core and dense
    """
    df = df.copy()
    
    # Apply user-defined constraints
    for choice in optim_paras["choices"]:
        df[choice] = False
    
    for choice in options[f"inadmissible_choices_{category}"].keys():
        for formula in options[f"inadmissible_choices_{category}"][choice]:
            try: 
                df[choice] |= df.eval(formula)
            except pd.core.computation.ops.UndefinedVariableError:
                pass
    
    # Dirty Fix to reverse False and True
    for choice in optim_paras["choices"]:
        df[choice] = df[choice].map(lambda x : False if x is True else True)
        
    return df

def _split_core_state_space(core, optim_paras):
    """
    This function splits the sp according to period and choice set
    """
    periodic_cores = {idx: sub for idx, sub in core.groupby("period")}
    periodic_choice_cores = {(period, choice_set): sub 
                           for period in periodic_cores
                           for choice_set, sub 
                           in periodic_cores[period].groupby(list(optim_paras["choices"]))
                          }
    
    return periodic_choice_cores


def _get_subspace_of_choice(choice_set):
    """
    This is WIP!
    """
    out = []
    choice_set = np.array(choice_set)
    check = np.where(np.invert(choice_set))
    
    for x in itertools.product([True, False],repeat=len(choice_set)):
        if all(np.all(y) == False for y in np.array(x)[check]):
            out.append(x)
        else:
            pass
    return out

    
def _split_full_state_space(period_choice_cores, dense_df, options, optim_paras):
    """
    This function should directly give us a number of indices after all!
    TODO: Major cleanup. Way to complicated for now! 
    """
    state_space = {}
    for (period, choice_set), df in period_choice_cores.items():
        for dense_state in dense_df.index:
            admissible_choices = _get_subspace_of_choice(
                dense_df.loc[dense_state,list(optim_paras["choices"].keys())].to_numpy())
            print(admissible_choices)
            if choice_set in admissible_choices: 
                df_ = df.copy().assign(
                    **dense_df.drop(
                        columns=optim_paras["choices"].keys()).loc[dense_state].to_dict())
                df_ = _create_choice_sets(
                    df_,
                    optim_paras,
                    options,
                    "mixed"
                )
               
                for choice_set_, sub_df in df_.groupby(list(optim_paras["choices"])):
                    #Here we want to assign an index
                    state_space[(period, choice_set_, dense_state)] = sub_df.index
    return state_space


### Ilustration of base objects

In [49]:
# We create the full core first! (Do we have to create covariates here?)
core, indexer = _create_core_and_indexer(optim_paras, options_)
    
# We apply all inadmissable states 
core = _create_choice_sets(core, optim_paras, options_, "core")
period_choice_cores = _split_core_state_space(core, optim_paras)

# Handle dense objects
#Create dense sp! TODO: Make less ugly!
dense_grid = rp.state_space._create_dense_state_space_grid(optim_paras)
dense_cov = rp.state_space._create_dense_state_space_covariates(dense_grid, optim_paras, options_)
dense_df = pd.DataFrame(dense_cov).transpose()
dense_df = _create_choice_sets(dense_df, optim_paras, options_, "dense")
    

period_choice_dense_cores = _split_full_state_space(period_choice_cores, dense_df, options_, optim_paras)
print(period_choice_dense_cores)

[(True, True, True), (True, True, False), (True, False, True), (True, False, False), (False, True, True), (False, True, False), (False, False, True), (False, False, False)]
[(True, True, True), (True, True, False), (True, False, True), (True, False, False), (False, True, True), (False, True, False), (False, False, True), (False, False, False)]
[(True, True, True), (True, True, False), (True, False, True), (True, False, False), (False, True, True), (False, True, False), (False, False, True), (False, False, False)]
[(True, True, True), (True, True, False), (True, False, True), (True, False, False), (False, True, True), (False, True, False), (False, False, True), (False, False, False)]
[(True, True, True), (True, True, False), (True, False, True), (True, False, False), (False, True, True), (False, True, False), (False, False, True), (False, False, False)]
[(True, True, True), (True, True, False), (True, False, True), (True, False, False), (False, True, True), (False, True, False), (False,

In [50]:
dense_df

,health,fishing,friday,hammock
0,0,True,True,True
1,1,True,True,True


## Check creation and reshaping of draws

In [51]:
# Create base draws
base_draws_sol = create_base_draws(
        (options_["n_periods"], options_["solution_draws"], len(optim_paras["choices"])),
        next(options_["solution_seed_startup"]),
        options_["monte_carlo_sequence"])

# Get period choice cores for one dense dim
period_choice_cores_draws = {key:period_choice_dense_cores[key] for key in period_choice_dense_cores.keys() if key[2]==(0,)}

# Reshape draws for that state space 
base_draws_sol_reshaped = _reshape_base_draws(base_draws_sol, period_choice_cores_draws, options)

plankreuz 1
plankreuz 1
plankreuz 1
plankreuz 1
plankreuz 1
plankreuz 1
plankreuz 1
plankreuz 1
plankreuz 1
plankreuz 1


In [52]:
period_choice_cores_draws

{(0, (True, True, True), (0,)): Int64Index([0], dtype='int64'),
 (1, (True, True, True), (0,)): Int64Index([1, 2], dtype='int64'),
 (2, (True, True, True), (0,)): Int64Index([3, 4, 5, 6], dtype='int64'),
 (3,
  (True, True, True),
  (0,)): Int64Index([7, 8, 9, 10, 11, 12, 13, 14], dtype='int64'),
 (4,
  (True, True, True),
  (0,)): Int64Index([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
             31],
            dtype='int64'),
 (5,
  (True, True, True),
  (0,)): Int64Index([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
             49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
            dtype='int64'),
 (6,
  (True, True, True),
  (0,)): Int64Index([ 61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
              74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
              87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
             100, 101, 102, 103, 104],
            dtype='int64'),
 (

## Plug things together!

Duplicate each part with all dense vectors and re-compute `is_inadmissible` with the dense information. This is necessary because observables may block a choice. This gives us the final combination between dense vectors, choice sets and parts of the core state space.

In [105]:
#TODO: Handle dense dims correctly! 
def create_sp():
    """
    Define a function that calls all the prior methods to create something 
    that looks like a sp! 
    """
    # We create the full core first! (Do we have to create covariates here?)
    core, indexer = _create_core_and_indexer(optim_paras, options_)
    
    #We apply all inadmissable states 
    core = _create_choice_sets(core, optim_paras, options, "core")
    period_choice_cores = _split_core_state_space(core)
    
    #Create dense sp! TODO: Make less ugly!
    dense_grid = rp.state_space._create_dense_state_space_grid(optim_paras)
    dense_cov = rp.state_space._create_dense_state_space_covariates(dense_grid, optim_paras, options_)
    dense_df = pd.DataFrame(dense_cov).transpose()
    dense_df = _create_choice_sets(dense_df, options, optim_paras, "dense")
    
    #Plug both together
    period_choice_dense_cores = _split_base_state_space(period_choice_cores,
                                                        dense_df,
                                                        options,
                                                        optim_paras
                                                        )
    
    
    #Create base draws sol
    base_draws_sol = create_base_draws(
        (options["n_periods"], options["solution_draws"], len(optim_paras["choices"])),
        next(options["solution_seed_startup"]),
        options["monte_carlo_sequence"],
    )
    
    if dense:
        state_space = _MultiDimStateSpace(
            core, period_choice_dense_cores, indexer, base_draws_sol, optim_paras, options, dense
        )
    else:
        state_space = _SingleDimStateSpace(
            core, indexer, base_draws_sol, optim_paras, options
        )

    return state_space
    
    

### The StateSpace

The idea is that the the state space has three levels: 
- _MultiDimStateSpace: This is the highest level. Takes the core, the indexer and the period_choice_dense core indexer. It then just creates one instance of the lower class for each dense covariate and distributes information accordingly. (Not a lot changes here)
-_SingleDimStateSpace: This state space manages one single core sp. 
It creates one instance of the lower class for each choice set that appears and distributes information accordingly. 
This object has a different role than before. Importantly some attributes are also changed by operations from the lower classes!
-ChoiceSetSpace:
This class is the lowest object. It receives the indices of core states that belong to each choice set in every period. 
Al information like wages is assigned to this lowest class since it is only processed there. For continuation values the class however draws on the SingleDimsSP since it is possible that child states have differet choice sets

In [27]:
#We try to get the easiest workflow!
#This will inplace change the cont value array
class _ChoiceSetSpace(_BaseStateSpace):
    """
    This is just a bundle of methods that should calculate 
    """
    def __init__(
    choice_set,
    period_cores,
    expected_value_functions, 
    base_draws_sol,
    optim_paras,
    options,
    indices_of_child_states
    ):
        self.dense = dense 
        self.choice_set = choice_set
        self.num_choices = choice_set.count(True)
        self.core_dict = core_dict
        self.base_draws_sol = base_draws_sol
        self.options = options
        self.optim_paras = optim_paras
        # We want this as pointer! WIll that work?
        self.expected_value_functions = value_functions
        self.indices_of_child_states
        

    def get_continuation_values(self, period, indices=None):
        """
        For now only works with explicit period! 
        """
        n_periods = len(self.indexer)

        if period == n_periods - 1:
            last_slice = self.period_cores[-1]
            n_states_last_period = len(last_slice)
            continuation_values = np.zeros((n_states_last_period, self.n_choices))

        else:
            child_indices = self.get_attribute("indices_of_child_states")[self.period_cores[period]][:,choice_set]

            mask = child_indices != INDEXER_INVALID_INDEX
            valid_indices = np.where(mask, child_indices, 0)
            continuation_values = np.where(
                mask, self.get_attribute("expected_value_functions")[valid_indices], 0
            )
        continuation_values[:,self.choice_set]
        
        return continuation_values
    
    def get_attribute(self,attr):
        return getattr(self, attr)
    
    def set_attribute(self,attr,value):
        self.get_attribute(attribute)[:] = value

        

NameError: name '_BaseStateSpace' is not defined

In [28]:
class _SingleDimsSpace(_BaseStateSpace):
    """
    We should initialize with child states!
    Cont values are something we need tp get the super class for!
    """

    def __init__(
        self,
        core,
        period_choice_cores,
        indexer,
        base_draws_sol,
        optim_para
        options,
        dense_dim=None,
        dense_covariates=None,
        is_inadmissible=None,
        indices_of_child_states=None,
        slices_by_periods=None,
    ):
        self.dense_dim = dense_dim
        self.core = core
        self.period_choice_cores = period_choice_cores 
        self.indexer = indexer
        self.dense_covariates = dense_covariates if dense_covariates is not None else {}
        self.mixed_covariates = options["covariates_mixed"]
        self.base_draws_sol = _reshape_base_draws(base_draws_sol, self.period_choice_cores)
        self._initialize_attributes(optim_paras)
        self.indices_of_child_states = (
            super()._create_indices_of_child_states(optim_paras)
            if indices_of_child_states is None
            else indices_of_child_states
        )
        # No Hotfix after all xD What is the problem with this way?
        self.expected_value_functions = np.empty(self.core.shape[0])
        
        
    def get_continuation_values(self,period):
        n_periods = len(self.indexer)
        choice_cores = {key[1]:self.period_choice_cores[key] if key[0]==period}
        continuation_values = dict()
        
        for choice_sets in choice_cores.keys():
            
            if period == n_periods - 1:
                last_slice = choice_cores[choice_set]
                n_states_last_period = len(last_slice)
                continuation_values_choice = np.zeros((n_states_last_period, self.n_choices))
            else:
                child_indices = self.get_attribute("indices_of_child_states")[choice_cores[choice_set],choice_set]
                mask = child_indices != INDEXER_INVALID_INDEX
                valid_indices = np.where(mask, child_indices, 0)
                continuation_values_choice = np.where(
                    mask, self.get_attribute("expected_value_functions")[valid_indices], 0
                )
            continuation_values[choice_set] = continuation_values_choice
        
        return continuation_values

        
    def get_attribute(self, attr):
        """Get an attribute of the state space."""
        return getattr(self, attr)

    
    def get_attribute_from_period_choice(self, attribute, period, choice):
        attr = self.get_attribute(attribute)
        slice_ = self.cores[(period, choice)]
        out = attr[slice_]
        return out
        
    def set_attribute(self, attribute, value):
        self.get_attribute(attribute)[:] = value
    
    def set_attribute_from_period_choice_set(self, attribute, value, period, choice):
        self.get_attribute_from_period_choice(attribute, period, choice)[:] = value
    
    @property
    def states(self):
        states = self.core.copy().assign(**self.dense_covariates)
        states = compute_covariates(states, self.mixed_covariates)
        return states



SyntaxError: invalid syntax (<ipython-input-28-0547c38d6f16>, line 14)

In [29]:
class _MultiDimStateSpace(_BaseStateSpace):
    """The state space of a discrete choice dynamic programming model.
    This class wraps the whole state space of the model.
    In this case we take the whole seperated choice space and create individual objects for each 
    dense option. Operations are still seperated and thus the old logic still largely applies. 
    Down the road we might want to redcue the number of input variables since this looks a bit confusing now 
    """

    def __init__(self, core, period_choice_dense_cores, indexer, base_draws_sol, optim_paras, options, dense):
        self.base_draws_sol = base_draws_sol
        self.core = core
        self.indexer = indexer
        self.is_inadmissible = super()._create_is_inadmissible(optim_paras, options)
        self.indices_of_child_states = super()._create_indices_of_child_states(
            optim_paras
        )
        self.period_choice_dense_cores = period_choice_dense_cores
        self.sub_state_spaces = {
            dense_dim: _SingleDimStateSpace(
                self.core,
                self.indexer,
                {key[:2]:self.period_choice_dense_cores[key] for key in self.period_choice_dense_cores.keys() if key[0]==dense_dim }
                self.base_draws_sol,
                optim_paras,
                options,
                dense_dim,
                dense_covariates,
                self.is_inadmissible,
                self.indices_of_child_states,
                self.slices_by_periods,
            )
            for dense_dim, dense_covariates in dense.items()
        }
        

    def get_attribute(self, attribute):
        return {
            key: sss.get_attribute(attribute)
            for key, sss in self.sub_state_spaces.items()
        }

    def get_attribute_from_choice_set_private(self, attribute, period):
                return {
            key: sss.set_attribute_choice_sets_private(attribute, period)
            for key, sss in self.sub_state_spaces.items()
        }
    
    def get_attribute_from_period_choice_set(self, attribute, period):
        return {
            key: sss.get_attribute_from_period(attribute, period)
            for key, sss in self.sub_state_spaces.items()
        }
    
    def get_continuation_values(self, period, indices=None):
        return {
            key: sss.get_continuation_values(period, indices)
            for key, sss in self.sub_state_spaces.items()
        }
    
    def set_attribute(self, attribute, value):
        for key, sss in self.sub_state_spaces.items():
            sss.set_attribute(attribute, value[key])

    
    def set_attribute_from_period_choice_set(self, attribute, period, choice_set, value):
        for key, sss in self.sub_state_spaces.items():
                sss.set_attribute_from_choice_set_private(attribute,choice_set,value[key])
        
        
    @property
    def states(self):
        return {key: sss.states for key, sss in self.sub_state_spaces.items()}

SyntaxError: invalid syntax (<ipython-input-29-5fcce86de101>, line 23)

We dont need the slices per period and is inadmissable anymore since this information is now already contained in the period_choice_dense_cores!

In [30]:
class _BaseStateSpace:
    """The base class of a state space.
    The base class includes some methods which should be available to both state spaces
    and are shared between multiple sub state spaces.
    """

    def _create_indices_of_child_states(self, optim_paras):
        """For each parent state get the indices of child states.
        During the backward induction, the ``expected_value_functions`` in the future
        period serve as the ``continuation_values`` of the current period. As the
        indices for child states never change, these indices can be precomputed and
        added to the state_space.
        Actually, the indices of the child states do not have to cover the last period,
        but it makes the code prettier and reduces the need to expand the indices in the
        estimation.
        """
        n_choices = len(optim_paras["choices"])
        n_choices_w_exp = len(optim_paras["choices_w_exp"])
        n_periods = optim_paras["n_periods"]
        n_states = self.core.shape[0]
        core_columns = create_core_state_space_columns(optim_paras)

        indices = np.full(
            (n_states, n_choices), INDEXER_INVALID_INDEX, dtype=INDEXER_DTYPE
        )

        # Skip the last period which does not have child states.
        for period in reversed(range(n_periods - 1)):
            states_in_period = self.core.query("period == @period")[
                core_columns
            ].to_numpy(dtype=np.int8)

            indices = _insert_indices_of_child_states(
                indices,
                states_in_period,
                self.indexer[period],
                self.indexer[period + 1],
                self.is_inadmissible,
                n_choices_w_exp,
                optim_paras["n_lagged_choices"],
            )

        return indices

### Base Draws  
Drawing and processing shocks is more difficult than in the case without choice_sets since we have to align draws with choice sets at some point. Two particular challenges arise: 
- If we stick to the current specification/interface we have to define a rule    according to which shocks are split between choice sets in each period
- Since period-choice sets differ between dense dims. Thus we can not draw one valid set of shocks that can be used by each state space. 

I think we should discuss whether we want to change specification of these shocks to align better with the new structure!

**Work Around**: 
We will draw the full np.array of shocks in the beginning and initialize the _MultiDimsStateSpace with this array. 
We will pass the array to each _SingleDimsStateSpace. 
There we will use the period_choice_cores (dict: (period,choice_set):corresponding core indices) to transform the np.array to a dict that contains a subset of the array for each period_choice combination. 
In each period we will distribute draws to choice_sets according to size. 
Thereafter we will attach the relevant section of the dict to each _ChoiceSetSpace!
Thus nothing in the create_base_draws function for now!

In [36]:
def _reshape_base_draws(draws,period_choice_cores,options):
    """
    Distribute draws across period choice core subsets. 
    Highly preliminary for illustrative purposes! 
    """
    size_dict = {key:len(period_choice_cores[key]) for key in period_choice_cores.keys()}
    n_states_draws = draws.shape[1]
    out = dict()
    for period in range(options["n_periods"]):
        period_draws = draws[period,:,:]
        prop_dict = {key:size_dict[key] for key in size_dict.keys() if key[0]==period}
        n_states_core = np.array(list(prop_dict.values())).sum()
        pos = 0
        for key in prop_dict.keys():
            num_states = int(np.ceil(prop_dict[key] * n_states_draws/n_states_core))
            if pos+num_states > n_states_draws:
                num_states = n_states_draw - pos
            out[key] = draws[period,slice(pos,pos+num_states),key[1]]
            pos = pos + num_states
    
    return out
    

In [32]:
np.ceil(3.4)

4.0

### Calculation of covariates
We want to adapt the calculation of covariates to our new structure
Instead of returning two numpy arrays for each dense dim we return a dict with all 
period, choice_set combinations with arrays of corresponding size for each dense dim. 


In [33]:
@parallelize_across_dense_dimensions
def _create_choice_rewards(states,dense_period_choice_cores, is_inadmissible, optim_paras):
    """Create wage and non-pecuniary reward for each state and choice.
    Note that missing wages filled with ones and missing non-pecuniary rewards with
    zeros. This is done in :meth:`_initialize_attributes`.
    """
    n_states = states.shape[0]
    n_choices = len(optim_paras["choices"])
    
    # Initialize objects
    out_wages = {
        (period,choice_set):np.ones(
        (len(period_choice_cores[(period,choice_set)]),
         choice_set.count(True)))
                }
    
    out_nonpecs = {
        (period,choice_set):np.zeros(
        (len(period_choice_cores[(period,choice_set)]),
         choice_set.count(True)))
                }
    
    for (period, choice_set) in period_choice_cores.keys():
        # Funktioniert Boolean indexing mit Listen sonst muss ich mir da noch was überlegen
        for i, choice in enumerate(optim_paras["choices"][choice_set]):
            # Get releavnt portion of the state space
            states_period_choice = states.loc[period_choice_cores[(period,choice_set)]]
            if f"wage_{choice}" in optim_paras:
                wage_columns = optim_paras[f"wage_{choice}"].index
                log_wage = np.dot(
                    states_period_choice[wage_columns].to_numpy(dtype=COVARIATES_DOT_PRODUCT_DTYPE),
                    optim_paras[f"wage_{choice}"].to_numpy(),
                )
                wages[:, i] = np.exp(log_wage)
                out_wages[(choice_set, period)] = wages 

            if f"nonpec_{choice}" in optim_paras:
                nonpec_columns = optim_paras[f"nonpec_{choice}"].index
                nonpecs[:, i] = np.dot(
                    states_period_choice[nonpec_columns].to_numpy(dtype=COVARIATES_DOT_PRODUCT_DTYPE),
                    optim_paras[f"nonpec_{choice}"].to_numpy(),
                )
                out_wages[(choice_set, period)] = nonpecs

    return out_wages, out_nonpecs

NameError: name 'parallelize_across_dense_dimensions' is not defined

### Solution

Then, we are ready to modify the solution.

1. Change `_create_choice_rewards` such that only the rewards of choices in the choice set are computed.
2. Implement a new `get_continuation_values` function for the `StateSpace`.
3. Run the backward induction.

In [ ]:
def solve(params, options, state_space):
    """Solve the model."""
    optim_paras, options = process_params_and_options(params, options)

    states = state_space.states
    dense_period_choice_cores = {
        key[:2]:self.period_choice_dense_cores[key] for key in self.period_choice_dense_cores.keys() if key[0]==dense_dim 
    }
    is_inadmissible = state_space.get_attribute("is_inadmissible")

    wages, nonpecs = _create_choice_rewards(states, state_space,is_inadmissible, optim_paras)
    state_space.set_attribute_from_choice_set_private("wages", wages)
    state_space.set_attribute_from_choice_set_private("nonpecs", nonpecs)

    state_space = _solve_with_backward_induction(state_space, optim_paras, options)

    return state_space

I think we would want to loop over all period, state, dense combs and calculate their corresponding continuation values!
How should we realize that. Would we want these simple states to have 

In [ ]:
def _solve_with_backward_induction(state_space, optim_paras, options):
    """Calculate utilities with backward induction.
    Parameters
    ----------
    state_space : :class:`~respy.state_space.StateSpace`
        State space of the model which is not solved yet.
    optim_paras : dict
        Parsed model parameters affected by the optimization.
    options : dict
        Optimization independent model options.
    Returns
    -------
    state_space : :class:`~respy.state_space.StateSpace`
    """
    n_wages = len(optim_paras["choices_w_wage"])
    n_periods = optim_paras["n_periods"]

    draws_emax_risk = transform_base_draws_with_cholesky_factor(
        state_space.base_draws_sol, optim_paras["shocks_cholesky"], n_wages
    )

    for period in reversed(range(n_periods)):
        n_core_states = state_space.core.query("period == @period").shape[0]

        wages = state_space.get_attribute_from_period("wages", period)
        nonpecs = state_space.get_attribute_from_period("nonpecs", period)
        continuation_values = state_space.get_continuation_values(period)
        period_draws_emax_risk = draws_emax_risk[period]

        # The number of interpolation points is the same for all periods. Thus, for
        # some periods the number of interpolation points is larger than the actual
        # number of states. In this case, no interpolation is needed.
        n_dense_combinations = len(getattr(state_space, "sub_state_spaces", [1]))
        n_states_in_period = n_core_states * n_dense_combinations
        any_interpolated = (
            options["interpolation_points"] <= n_states_in_period
            and options["interpolation_points"] != -1
        )

        # Handle myopic individuals.
        if optim_paras["delta"] == 0:
            if hasattr(state_space, "sub_state_spaces"):
                period_expected_value_functions = {
                    dense_idx: 0 for dense_idx in state_space.sub_state_spaces
                }